In [1]:
#pip install numpy==1.26.4

In [2]:
import pandas as pd
import numpy as np


In [3]:
import pytz
from datetime import datetime, timedelta
d = datetime.now(pytz.timezone('Asia/Calcutta'))
d = d.strftime('%Y%m%d').upper()

d = '20260202'
version = 'eum' #amm

In [4]:
file = 'workbench_data_' + version + 'ocp_wPO_' + d
# print(file)
data = pd.read_csv(r'E:\_Projects\_outputs\orders\snd\\' + file + '_v2.csv', low_memory=False)

In [5]:
data_ex = data

# Filter Ship Date - 6 months
# data_ex['Ship Date'] = pd.to_datetime(data_ex['Ship Date'], format='%d-%m-%Y')
data_ex['Ship Date'] = pd.to_datetime(data_ex['Ship Date'], format='%Y-%m-%d')
data_ex['date_6m'] = pd.to_datetime('today').date() + timedelta(days=30*6)
data_ex['date_6m'] = data_ex['date_6m'].astype('datetime64[ns]')

# Filter Supply Type - Planned Buy/Make, IR No not null
condtn1 = (data_ex['Supply Type'] == 'Plnd-Buy') 
condtn2 = (data_ex['Supply Type']  == 'Plnd-Make')
condtn3 = (data_ex['IR No.'].isnull())
condtn4 = (data_ex['Ship Date'] < data_ex['date_6m'])
# condtn4 = (data_ex['Ship Date'] <= (data_ex['Ship Date'] + pd.DateOffset(months=6)))
data_ex = data_ex[( condtn1 | (condtn2 & condtn3) ) & condtn4].copy()

data_ex.drop(columns={'date_6m'}, axis=1, inplace=True)
# data_ex.to_csv(r'\\dayorg1\orgshare\TEAMS\ERP Shared Folder\Global S&OP\workbench\ex\workbench_ex_' + d + '.csv', index=False)
data_ex.to_csv(r'E:\_Projects\_outputs\orders\snd\workbench_ex_' + d + '_1.csv', index=False)

In [6]:
data_ex.head()
#data_ex.shape

,UID,IID,Demand,Region,SO Ship Org,Org Code,Area,Org Name,DEMAND VAL,Line Number,...,SO Valid_to_Ship++,SO Val_to_Ship By,SO Val_to_Ship TS,MS CTB Status++,MS CTB Reason,MS Proposed SSD,MS CTB Comments,MS CTB TS,MS CTB By,Preferred Offer
0,IPPWOTNS707729|CL|16161|8.1,IPPWOTNS707729|,Order,CLA,DUB,CL,Chile Peru,Chile Inventory,16161.0,8.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IPPWOTNS707730|CL|16161|9.1,IPPWOTNS707730|,Order,CLA,DUB,CL,Chile Peru,Chile Inventory,16161.0,9.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IPPWOTNS707721|CL|16288|10.1,IPPWOTNS707721|,Order,CLA,DUB,CL,Chile Peru,Chile Inventory,16288.0,10.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IPPWOTNS707722|CL|16288|11.1,IPPWOTNS707722|,Order,CLA,DUB,CL,Chile Peru,Chile Inventory,16288.0,11.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IPPWOT680875|CL|16519|10.1,IPPWOT680875|,Order,CLA,DUB,CL,Chile Peru,Chile Inventory,16519.0,10.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Update Zone
zone_map = pd.read_csv(r'E:\_Projects\lookups\country_zone_region.csv', \
           encoding='utf-8')

In [8]:
data = pd.merge(data, zone_map, left_on='Org Code', right_on='country', how='left', validate='m:1')

In [9]:
#data.shape

In [10]:
data['Parent'] = np.where(data['PID'].str.contains('MC'), data['PID'].str[:6], '')
data['Parent'] = np.where( (data['PID'].str.contains('M')==True) &  \
                           (data['PID'].str.contains('C')==False) & \
                           (data['Parent']==''), \
                          data['PID'].str[:5], data['Parent'])


In [11]:
rows_ex_inv = ['Region','Area','Org Name','Org Code','LOB','Key Account','Customer_End Customer','Product Range',\
          'Class','Data Series','PID','PID Type','zone','Parent']

In [12]:
data_ex2 = data

# Filter SAD - 1 year
# data_ex2['SAD'] = pd.to_datetime(data_ex2['SAD'], format='%d-%m-%Y')
data_ex2['SAD'] = pd.to_datetime(data_ex2['SAD'], format='%Y-%m-%d')
data_ex2['date_12m'] = pd.to_datetime('today').date() + timedelta(days=360)
data_ex2['date_12m'] = data_ex2['date_12m'].astype('datetime64[ns]')


# Filter for Validation Flag, PID Type
condtn1a = (data_ex2['Validation Flag']=='New/Rel PO')
condtn2a = (data_ex2['PID Type'].isin(['M','U']))
# condtn3a = (data_ex2['SAD'] + pd.DateOffset(months=12))


condtn3a = (data_ex2['SAD'] < data_ex2['date_12m'])

#data_ex2 = data_ex2[condtn1a & condtn2a & condtn3a].copy() 


data_ex2 = data_ex2[condtn1a & condtn2a].copy() 


data_ex2.drop(columns={'date_12m'}, axis=1, inplace=True)

# SAD month
data_ex2['SAD_mon'] = pd.to_datetime(data_ex2['SAD'], errors='coerce').dt.strftime('%Y%m')
data_ex2['Data Series'] = 'ex'

data_ex2['Key Account'] = data_ex2['Key Account'].fillna('OTHER')
data_ex2.to_csv(r'E:\_Projects\_outputs\orders\snd\Validate_Ex.csv', index=True)

# Pivot
data_ex2_pvt = data_ex2.pivot_table(index=rows_ex_inv, columns=['SAD_mon'], values=['Supply Qty'], aggfunc='sum')
#margins=True, margins_name='Total'

data_ex2_pvt.to_csv(r'E:\_Projects\_outputs\orders\snd\workbench_ex_' + d + '.csv', index=True)
#data_ex2

In [13]:
data_inv = data

# Filter Supply Type, Demand
sply_type_lst = ['OH RS', 'OH', 'IT'] 
condtn1b = data_inv['Supply Type'].isin(sply_type_lst)
condtn2b = data_inv['Demand'].str.contains('WIP','PO-')
condtn2c = (data_inv['PID Type'].isin(['M','U']))
data_inv = data_inv[condtn1b & ~condtn2b & condtn2c].copy()

# SAD month
data_inv['SAD_mon'] = pd.to_datetime(data_inv['SAD'], errors='coerce').dt.strftime('%Y%m')
data_inv['Data Series'] = 'Neg Inv'
data_inv['Key Account'] = data_inv['Key Account'].fillna('OTHER')
data_inv.to_csv(r'E:\_Projects\_outputs\orders\snd\Validate_Inv.csv', index=True)
# Pivot
data_inv_pvt = data_inv.pivot_table(index=rows_ex_inv, columns=['SAD_mon'], values=['Supply Qty'], aggfunc='sum')

# data_inv_pvt.to_csv(r'\\dayorg1\orgshare\TEAMS\ERP Shared Folder\Global S&OP\Supply Plan\Other_Data\workbench_inv_' + d + '.csv', index=True)
data_inv_pvt.to_csv(r'E:\_Projects\_outputs\orders\snd\workbench_inv_' + d + '.csv', index=True)